# Functions
 1. API Reqest to Place a phone call  (placePhoneCall)
 2. API Request to Send Message (sendSMS)
 3. API Request to get the logs of all Calls placed 
 4. Report generation
 5. Fuction to  send emails
 6. Function to generate the Excel File 
 
 
 		<SMTP_SERVER>gsmtp.mxns.loc</SMTP_SERVER>
		<SMTP_SERVER_PORT>25</SMTP_SERVER_PORT>
		<SMTP_CONNEXION_TIMEOUT>30</SMTP_CONNEXION_TIMEOUT>
		<SMTP_EMAIL_SENDER>COAresults.na@mxns.com</SMTP_EMAIL_SENDER>
		<SMTP_EMAIL_REPLYTO>midwest.region@mxns.com</SMTP_EMAIL_REPLYTO>

In [4]:
# Imports 
from twilio.rest import Client
import sqlalchemy as sql
import pandas as pd
import pytz
import numpy as np
import logging
from urllib.parse import urlencode, quote_plus
from datetime import datetime, timedelta
import json
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from win32com.client import Dispatch

In [7]:
#Static Credentials
twilioAccountId='AC48441c785c05232da5eca8fc91f9b26b'
twilioAccountKey='bf09ec57921a229322d60464f4c4d9ca'
twilioAPIUrl=''
dbHost='mxnunx41'
dbPort='1521'
dbUser='lims'
dbPassword='Mumwit63'
dbName='hwi1'
dburl='oracle://'
stmpEmail=''
emailFromAddress=''
twilioPhoneNumber='+13127577804'
customerPhNumber='+12818440171'
lastSuccessfulExecution=''
filePath='C:\\Users\\agundlapalli\\PycharmProjects\\JupyterNotebooks\\FilesToSend\\'
acrivedFilePath=''
disclaimerText='Such results are intended for use by persons having professional skill and training in the interpretation of microbiological testing results.  Mérieux NutriScience assumes no responsibility, and you hereby waive all claims against Mérieux NutriScience, for interpretation of such results.'
dt1 = datetime.now()

In [3]:
#send Email to the Contact Role persons if Email alert is enabled
def sendEmailContactRolePersons(dfContactRole,clientReportColums,dfOOSData,fileNameTS,filePath):
    for index, row in dfContactRole.iterrows():
        contact_role = row['contact_role']
        contactRoleQuery = "select p.name,p.phone,p.email_addr,p.company from contact_role_persons crp , person p where crp.person = p.name and "
        contactRoleQuery = contactRoleQuery + "(crp.x_enable_chem_oos = 'T' OR crp.x_enable_qual_oos = 'T' or crp.x_enable_quan_oos = 'T' or crp.x_enable_dairy_oos = 'T') and contact_role='" + contact_role + "'"
        cRoleData = getDataFromDB(contactRoleQuery,'Email Contact Role Query')
        if (len(cRoleData.index) != 0):
            df3 = dfOOSData.loc[dfOOSData['contact_role'] == contact_role, clientReportColums]
            fname = generateReportFiles(df3, contact_role,fileNameTS,filePath)
            emailAddressList = cRoleData['email_addr'].tolist()
            #sendEmailAttachment(emailAddressList,fname)

In [4]:
# Sends Email to the respective Program Person if Email alert is enabled
def sendEmailProgramPersons(dfProgram,clientReportColums,dfOOSData,fileNameTS,filePath):
    for index, row in dfProgram.iterrows():
        program = row['program']
        programPersonQuery = "select p.name,p.phone,p.email_addr,p.company from program_persons pp ,person p where pp.person = p.name and "
        programPersonQuery = programPersonQuery + "(pp.x_enable_chem_oos = 'T' OR pp.x_enable_qual_oos = 'T' OR pp.x_enable_quan_oos = 'T' OR pp.x_enable_dairy_oos='T') and pp.program='" + program + "'"
        programPData = getDataFromDB(programPersonQuery,'Email Program Query')
        if (len(programPData.index) != 0):
            df3 = dfOOSData.loc[dfOOSData['program'] == program, clientReportColums]
            fname = generateReportFiles(df3, program, fileNameTS, filePath)
            emailAddressList = programPData['email_addr'].tolist()
            # sendEmailAttachment(emailAddressList,fname)

In [118]:
# Sends Automated Call to the respective Contact Role persons if Automated Phone alert is enabled
def sendAutomatedPhoneContactRolePersons(dfContactRole,dfOOSData,clientReportColumnsPhone):
    for index, row in dfContactRole.iterrows():
        cnt = 1
        contact_role = row['contact_role']
        contactRoleQuery = "select p.alt_phone,p.company,p.first_name,p.last_name,p.city , p.state , le.value from contact_role_persons crp , person p , list_entry le where crp.person = p.name and p.state = le.name and le.list='STATES' and"
        contactRoleQuery = contactRoleQuery + "(crp.x_action_chem_oos = 'T' OR crp.x_action_qual_oos = 'T' or crp.x_action_quan_oos = 'T' or crp.x_action_dairy_oos = 'T') and contact_role='" + contact_role + "'"
        cRoleData = getDataFromDB(contactRoleQuery,'Phone Contact Role Query')
        if (len(cRoleData.index) != 0):
            callcnt = 1
            dfPhoneCallData = dfOOSData.loc[dfOOSData['contact_role'] == contact_role, clientReportColumnsPhone]
            dfPhoneCallData = dfPhoneCallData.reset_index(drop=True)
            print(contact_role)
            # print(dfPhoneCallData)
            if (len(dfPhoneCallData.index) > 1):
                cnt = 2
            for crdIndex, crdRow in cRoleData.iterrows():
                msgData = buildPhoneMsgData(dfPhoneCallData)
                msgData['fName'] = crdRow['first_name']
                msgData['lName'] = crdRow['last_name']
                msgData['companyName'] = crdRow['company']
                msgData['locationName'] = crdRow['city'] + "," + crdRow['value']
                callMsgData = cleanDict(msgData)
                phNumber = crdRow['alt_phone']
                #placePhoneCall1(callMsgData, phNumber, cnt)
                #placePhoneCall(callMsgData,phNumber,cnt)

In [119]:
def sendAutomatedPhoneProgramPersons(dfProgram,dfOOSData,clientReportColumnsPhone):
    for index, row in dfProgram.iterrows():
        cnt = 1
        program = row['program']
        programPersonQuery = "select p.alt_phone,p.company,p.first_name,p.last_name,p.city , p.state , le.value from program_persons pp ,person p ,list_entry le where pp.person = p.name and p.state = le.name and le.list='STATES' and"
        programPersonQuery = programPersonQuery + "(pp.x_action_chem_oos = 'T' OR pp.x_action_qual_oos = 'T' OR pp.x_action_quan_oos = 'T' OR pp.x_action_dairy_oos='T') and pp.program=" + program + "'"
        programPData = getDataFromDB(programPersonQuery,'Phone Program Query')
        if (len(programPData.index) != 0):
            callcnt = 1
            dfPhoneCallData = dfOOSData.loc[dfOOSData['program'] == program, clientReportColumnsPhone]
            dfPhoneCallData = dfPhoneCallData.reset_index(drop=True)
            # print(dfPhoneCallData)
            if (len(dfPhoneCallData.index) > 1):
                cnt = 2
            for crdIndex, pdRow in programPData.iterrows():
                msgData = buildPhoneMsgData(dfPhoneCallData)
                msgData['fName'] = pdRow['first_name']
                msgData['lName'] = pdRow['last_name']
                msgData['companyName'] = pdRow['company']
                msgData['locationName'] = pdRow['city'] + "," + pdRow['value']
                callMsgData = cleanDict(msgData)
                #phNumber = pdRow['alt_phone']
                # placePhoneCall1(callMsgData,phNumber,cnt)
                #added for testing purpose
                phNumber = random.choice(randomPhList)
                #placePhoneCall(callMsgData, phNumber, cnt)

In [120]:
# Sends Automated Text  to the respective Contact Role persons if Automated Text alert is enabled
def sendAutomatedTextContactRolePersons(dfContactRole,dfOOSData,clientReportColumnsText):
    for index, row in dfContactRole.iterrows():
        contact_role = row['contact_role']
        contactRoleQuery = "select p.alt_phone,p.company,p.first_name,p.last_name,p.city , p.state , le.value from contact_role_persons crp , person p , list_entry le where crp.person = p.name and p.state = le.name and le.list='STATES' and"
        contactRoleQuery = contactRoleQuery + "(crp.x_action_chem_oos = 'T' OR crp.x_action_qual_oos = 'T' or crp.x_action_quan_oos = 'T' or crp.x_action_dairy_oos = 'T') and contact_role='" + contact_role + "'"
        cRoleData = getDataFromDB(contactRoleQuery,'Text Contact Role Query')
        if (len(cRoleData.index) != 0):
            dfPhoneCallData = dfOOSData.loc[dfOOSData['contact_role'] == contact_role, clientReportColumnsText]
            dfPhoneCallData = dfPhoneCallData.reset_index(drop=True)
            print(contact_role)
            # print(dfPhoneCallData)
            c=1
            for index, row in dfPhoneCallData.iterrows():
                textMsgData=buildTextMsg(row)
                for crdIndex, crdRow in cRoleData.iterrows():
                    #generate msg data for SMS
                    #phNumber = crdRow['alt_phone']
                    #phNumber = '+12818440171'
                    phNumber = random.choice(randomPhList)
                    if(c==1):
                        sendSMS(textMsgData,phNumber)
                    c=c+1

In [121]:
# Sends Automated Text  to the respective Program persons if Automated Text alert is enabled
def sendAutomatedTextProgramPersons(dfContactRole,dfOOSData,clientReportColumnsText):
    for index, row in dfContactRole.iterrows():
        program = row['program']
        programPersonQuery = "select p.alt_phone,p.company,p.first_name,p.last_name,p.city , p.state , le.value from program_persons pp ,person p ,list_entry le where pp.person = p.name and p.state = le.name and le.list='STATES' and"
        programPersonQuery = programPersonQuery + "(pp.x_action_chem_oos = 'T' OR pp.x_action_qual_oos = 'T' OR pp.x_action_quan_oos = 'T' OR pp.x_action_dairy_oos='T') and pp.program=" + program + "'"
        programPData = getDataFromDB(programPersonQuery,'Text Program Query')
        if (len(programPData.index) != 0):
            dfPhoneCallData = dfOOSData.loc[dfOOSData['program'] == program, clientReportColumnsText]
            dfPhoneCallData = dfPhoneCallData.reset_index(drop=True)
            print(program)
            # print(dfPhoneCallData)
            for index, row in dfPhoneCallData.iterrows():
                textMsgData=buildTextMsg(row,program)
                for crdIndex, pdRow in programPData.iterrows():
                    #generate msg data for SMS
                    #phNumber = crdRow['alt_phone']
                    phNumber = '+12818440171'
                    #sendSMS(textMsgData,phNumber)

In [122]:
# Function to  Clean dict
def cleanDict(msgData):
    result = {}
    for key, value in msgData.items():
        if value !=None:
            result[key] = value
    return result

In [143]:
# Funciton to build dict for Phone call
def buildPhoneMsgData(dfPhoneCallData):
    retVal={}
    retVal['labName']=dfPhoneCallData.iloc[0]['lab_owner_loc']
    retVal['testName']=dfPhoneCallData.iloc[0]['phtestname']
    retVal['result']=dfPhoneCallData.iloc[0]['result_desc']
    retVal['unit']=dfPhoneCallData.iloc[0]['unit_desc']
    retVal['desc1']=dfPhoneCallData.iloc[0]['sample_desc1']
    retVal['desc2']=dfPhoneCallData.iloc[0]['sample_desc2']
    retVal['desc3']=dfPhoneCallData.iloc[0]['sample_desc3']
    retVal['desc4']=dfPhoneCallData.iloc[0]['sample_desc4']
    retVal['desc5']=dfPhoneCallData.iloc[0]['sample_desc5']
    retVal['desc6']=dfPhoneCallData.iloc[0]['sample_desc6']
    return retVal

In [124]:
# Funciton to build dict for Text
def buildTextMsg(row):
    msg=''
    msg = "From :MXNS(Do not reply)\nTest: "
    msg = msg +xstr(row['reported_name'])+ "\nResult: "
    msg = msg +xstr(row['result_desc']) + " "
    msg = msg +xstr(row['unit_desc']) + " "
    msg = msg +"\nMXNS ID :"+xstr(row['sample_number'])+ "\nSample Desc: "
    msg = msg +xstr(row['sample_desc1'])
    msg = msg +xstr(row['sample_desc2'])
    msg = msg +xstr(row['sample_desc3'])
    msg = msg +xstr(row['sample_desc4'])
    msg = msg +xstr(row['sample_desc5'])
    msg = msg +xstr(row['sample_desc6'])
    msg = msg + "\nFor all results, please click https://www.mymxns.com/#"
    return msg

In [125]:
# Functionto return null if None Type
def xstr(s):
    if s is None:
        return ''
    return str(s)+" "

In [126]:
# Place Call API
def placePhoneCall(msgData, customerPhNumber, cnt):
    logging.info('Function :  placePhoneCall1')
    if(cnt==1):
        urlBase='https://handler.twilio.com/twiml/EH8429701e067c1cf157248958910bf4ca?'
    else:
        urlBase='https://handler.twilio.com/twiml/EH60bf2e7c1926cdaca8ff282cf971cb7c?'
    twimlURL=''
    '''
    for key, value in msgData.items():
        twimlURL = twimlURL + key+ '='+URLEncoder.encode(value)+'&'
    twimlURL = twimlURL[:-1]
    '''
    twimlURL=urlencode(msgData, quote_via=quote_plus)
    twiMLBinURL=urlBase+twimlURL
    print(twiMLBinURL)
    # Api Call twilio
    client = Client(twilioAccountId,twilioAccountKey)
    call = client.calls.create(
        machine_detection='DetectMessageEnd',
        url=twiMLBinURL,
        to=customerPhNumber,
        from_='+13129385359'
    )
    #logging.info('Call Placed url'+ twiMLBinURL)

In [127]:
# Send SMS API
def sendSMS(smsMsgText,customerPhNumber):
    client = Client(twilioAccountId,twilioAccountKey)
    message = client.messages.create(
                     body=smsMsgText,
                     from_='+13127577804',
                     to=customerPhNumber)
    print("SMS sent to"+customerPhNumber)
    #logging.info('SMS Sent to ' + customerPhNumber)

In [128]:
# get Data from DB
def getDataFromDB(queryText,qType):
    logging.info('Fucntion : getDataFromDB')
    dburl = 'oracle://'+dbUser+':'+dbPassword+'@'+dbHost+':'+dbPort+'/'+dbName
    #print(dburl)
    engine=sql.create_engine(dburl)
    conn = engine.connect()
    #results = conn.execute(queryText)
    df = pd.read_sql(queryText, engine)
    #logging.info('Fucntion : getDataFromDB : Query Executed '+qType)
    return df

In [129]:
#get Call Log status
def getCallLogStatus(dt1):
    callList=[]
    #callDict={}
    client = Client(twilioAccountId, twilioAccountKey)
    calls = client.calls.list(start_time_before=datetime(dt1.year, dt1.month, dt1.day, dt1.hour, dt1.minute),
                              start_time_after=datetime(dt1.year, dt1.month, dt1.day, dt1.hour - 1, dt1.minute))
    for each in calls:
        callDict={}
        callDict['sid']=each.sid
        callDict['answered_by']=each.answered_by
        callDict['duration']=each.duration
        callDict['status']=each.status
        callDict['to']=each.to
        callDict['date_updated']=each.date_updated
        callDict['end_time']=each.end_time
        callDict['direction']=each.direction
        callList.append(callDict)
    dfCallLogs = pd.DataFrame(callList)
    dfCallLogs = dfCallLogs.loc[dfCallLogs['direction'] != 'inbound']
    return dfCallLog

In [130]:
# Function to generate files
def generateReportFiles(df,crole_program,fileNameTS,filePath):
    fileName='MXNS Out of Spec '+fileNameTS+' '+crole_program+'.xlsx'
    fileNamePath = filePath+fileName
    writer = pd.ExcelWriter(fileNamePath,engine='xlsxwriter')
    df.to_excel(writer,index=False ,sheet_name='OOS')
    workbook = writer.book
    worksheet = writer.sheets['OOS']
    #header_fmt = workbook.add_format({'bold': True})
    #worksheet.set_row(0, None, header_fmt)
    writer.save()
    return fileNamePath

In [131]:
# Send Email with attachment
def sendEmailAttachment(toaddr,fname):
    toaddr=["abhinav.gundlapalli@mxns.com","agundlapalli@uh.edu"]
    msg = MIMEMultipart()
    msg['From'] = 'reportingresults2.na@mxns.com'
    msg['Subject'] = "OOS Enhance Test"
    body = "Hello this is a test message"
    msg.attach(MIMEText(body, 'plain'))
    #filename = ""
    #attachment = open("PATH OF THE FILE", "rb")
    #part = MIMEBase('application', 'octet-stream')
    #part.set_payload((attachment).read())
    #encoders.encode_base64(part)
    #part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    #msg.attach(part)
    server = smtplib.SMTP('gsmtp.mxns.loc',25)
    text = msg.as_string()
    server.sendmail(msg['From'], toaddr, text)
    server.quit()

## OOS data function Process

In [132]:
#time Zones for running 
now_utc = dt.datetime.now(pytz.timezone('UTC'))
from_utc = now_utc - dt.timedelta(hours=1)
now_PST = now_utc.astimezone(pytz.timezone('US/Pacific')).strftime("%d-%b-%Y %I:%M:%S %p")
now_EST = now_utc.astimezone(pytz.timezone('US/Eastern')).strftime("%d-%b-%Y %I:%M:%S %p")
now_CST = now_utc.astimezone(pytz.timezone('US/Central')).strftime("%d-%b-%Y %I:%M:%S %p")
from_PST = from_utc.astimezone(pytz.timezone('US/Pacific')).strftime("%d-%b-%Y %I:%M:%S %p")
from_EST = from_utc.astimezone(pytz.timezone('US/Eastern')).strftime("%d-%b-%Y %I:%M:%S %p")
from_CST = from_utc.astimezone(pytz.timezone('US/Central')).strftime("%d-%b-%Y %I:%M:%S %p")
fileNameTS=now_utc.astimezone(pytz.timezone('US/Central')).strftime("%Y_%m_%d %H_%M hrs")

In [133]:
# OOS Query to getData
oosDataQuery="select * from oosData_view where time_zone = 'CST' AND " 
oosDataQuery = oosDataQuery + "test_reviewed >= to_date('"+from_CST+"', 'DD-MON-YYYY HH:MI:SS AM') AND "
oosDataQuery = oosDataQuery + "test_reviewed <= to_date('"+now_CST+"', 'DD-MON-YYYY HH:MI:SS AM') UNION "
oosDataQuery = oosDataQuery + "select * from oosData_view where time_zone = 'PST' AND "
oosDataQuery = oosDataQuery + "test_reviewed >= to_date('"+from_PST+"', 'DD-MON-YYYY HH:MI:SS AM') AND "
oosDataQuery = oosDataQuery + "test_reviewed <= to_date('"+now_PST+"', 'DD-MON-YYYY HH:MI:SS AM') UNION "
oosDataQuery = oosDataQuery + "select * from oosData_view where time_zone = 'EST' AND "
oosDataQuery = oosDataQuery + "test_reviewed >= to_date('"+from_EST+"', 'DD-MON-YYYY HH:MI:SS AM') AND "
oosDataQuery = oosDataQuery + "test_reviewed <= to_date('"+now_EST+"', 'DD-MON-YYYY HH:MI:SS AM')"
oosDataQuery
#dfOOSData=getDataFromDB(oosDataQuery)
#oosDataQuery="select * from oosData_view where time_zone = 'CST' AND test_reviewed >= to_date('20-Sep-2018 11:00:00 AM', 'DD-MON-YYYY HH:MI:SS AM') AND test_reviewed <= to_date('20-Sep-2018 12:00:00 PM', 'DD-MON-YYYY HH:MI:SS AM') UNION select * from oosData_view where time_zone = 'PST' AND test_reviewed >= to_date('20-Sep-2018 09:00:46 AM', 'DD-MON-YYYY HH:MI:SS AM') AND test_reviewed <= to_date('20-Sep-2018 10:00:46 AM', 'DD-MON-YYYY HH:MI:SS AM') UNION select * from oosData_view where time_zone = 'EST' AND test_reviewed >= to_date('20-Sep-2018 12:00:46 PM', 'DD-MON-YYYY HH:MI:SS AM') AND test_reviewed <= to_date('20-Sep-2018 01:00:46 PM', 'DD-MON-YYYY HH:MI:SS AM')"
dfOOSData=getDataFromDB(oosDataQuery,'OOS Data Query')

In [134]:
dfOOSData['phtestname'] = np.where((dfOOSData['e_reported_name'] == dfOOSData['reported_name']),
                                            dfOOSData['e_reported_name'],
                                            (dfOOSData['e_reported_name'] + " " + dfOOSData['reported_name']))
dfContactRoleUniqe = dfOOSData.drop_duplicates(subset=['contact_role'])
dfProgramUniqe = dfOOSData.drop_duplicates(subset=['program'])
dfContactRole = dfContactRoleUniqe[['contact_role']]

In [138]:
clientReportColums = ['sample_number', 'analysis','replicate_count','reported_name','result_desc','in_spec','min_limit','unit','unit_desc',
                     'sample_desc4','sample_desc5','sample_desc6','sample_desc7','sample_desc9','sample_desc10','sample_desc12',
                     'sample_desc13','test_location','lab']

    # Cloumns to generate the  Phone msg
clientReportColumnsPhone = ['sample_number', 'sample_desc1', 'sample_desc2', 'sample_desc3', 'sample_desc4',
                                'sample_desc5', 'sample_desc6', 'result_desc', 'unit_desc', 'lab_owner_loc',
                                'phtestname']
    # Cloumns to generate the  Text msg
clientReportColumnsText = ['sample_number', 'sample_desc1', 'sample_desc2', 'sample_desc3', 'sample_desc4',
                               'sample_desc5', 'sample_desc6', 'result_desc', 'unit_desc', 'reported_name']

In [139]:
dfOOSData.head()

,sample_number,test_completed,test_reviewed,contact_role,program,project,analysis,reported_name,e_reported_name,testrepname,...,sample_desc5,sample_desc6,sample_desc7,sample_desc8,sample_desc9,sample_desc10,sample_desc13,sample_desc14,sample_desc19,phtestname
0,381279598,2018-10-03 23:51:21,2018-10-04 12:13:40,HILA-QA-3,HILA-20,HILA-QA-3-CHG-1800089,LSP-FCN9P2,Genus Listeria,Genus Listeria - ELFA,Genus Listeria - ELFA,...,None,None,None,None,None,None,None,None,None,Genus Listeria - ELFA Genus Listeria
1,381279605,2018-10-03 23:51:59,2018-10-04 12:13:40,HILA-QA-3,HILA-20,HILA-QA-3-CHG-1800089,LSP-FCN9P2,Genus Listeria,Genus Listeria - ELFA,Genus Listeria - ELFA,...,None,None,None,None,None,None,None,None,None,Genus Listeria - ELFA Genus Listeria
2,381339170,2018-10-04 08:56:04,2018-10-04 09:52:47,CALI-TU-31,CAL-005,CALI-TU-31-TUL-1802439,YM,Mold,Yeast and Mold,Yeast and Mold,...,25 KG,LH NFDM,Export,None,None,None,None,None,None,Yeast and Mold Mold
3,381339176,2018-10-04 08:56:05,2018-10-04 09:52:47,CALI-TU-31,CAL-005,CALI-TU-31-TUL-1802439,YM,Mold,Yeast and Mold,Yeast and Mold,...,25 KG,LH NFDM,Export,None,None,None,None,None,None,Yeast and Mold Mold
4,381345059,2018-10-04 09:24:32,2018-10-04 09:31:32,CALI-TU-94,CALI-001,CALI-TU-94-TUL-1803466,YM,Mold,Yeast and Mold,Yeast and Mold,...,25 KG,LH NFDM,Export,None,None,None,None,None,None,Yeast and Mold Mold


In [140]:
oosDataQuery

"select * from oosData_view where time_zone = 'CST' AND test_reviewed >= to_date('04-Oct-2018 11:26:16 AM', 'DD-MON-YYYY HH:MI:SS AM') AND test_reviewed <= to_date('04-Oct-2018 12:26:16 PM', 'DD-MON-YYYY HH:MI:SS AM') UNION select * from oosData_view where time_zone = 'PST' AND test_reviewed >= to_date('04-Oct-2018 09:26:16 AM', 'DD-MON-YYYY HH:MI:SS AM') AND test_reviewed <= to_date('04-Oct-2018 10:26:16 AM', 'DD-MON-YYYY HH:MI:SS AM') UNION select * from oosData_view where time_zone = 'EST' AND test_reviewed >= to_date('04-Oct-2018 12:26:16 PM', 'DD-MON-YYYY HH:MI:SS AM') AND test_reviewed <= to_date('04-Oct-2018 01:26:16 PM', 'DD-MON-YYYY HH:MI:SS AM')"

In [12]:
callList=[]
#callDict={}
client = Client(twilioAccountId, twilioAccountKey)
calls = client.calls.list(start_time_before=datetime(dt1.year, dt1.month, dt1.day, dt1.hour, dt1.minute),
                              start_time_after=datetime(dt1.year, dt1.month, dt1.day-1, (dt1.hour - 1), dt1.minute))

In [13]:
calls

[<Twilio.Api.V2010.CallInstance account_sid=AC48441c785c05232da5eca8fc91f9b26b sid=CA20c550d0ca2b8c73e15d1d991805721f>,
 <Twilio.Api.V2010.CallInstance account_sid=AC48441c785c05232da5eca8fc91f9b26b sid=CA15a8f261e0d3b9addefba0832d98d885>,
 <Twilio.Api.V2010.CallInstance account_sid=AC48441c785c05232da5eca8fc91f9b26b sid=CA3acaca67e52c3e0be3aca5fba3ab0b28>,
 <Twilio.Api.V2010.CallInstance account_sid=AC48441c785c05232da5eca8fc91f9b26b sid=CA1a5c4c0868e872c27fa1c41fb28bbac8>,
 <Twilio.Api.V2010.CallInstance account_sid=AC48441c785c05232da5eca8fc91f9b26b sid=CA7501765ed631f000261280e2765633f6>,
 <Twilio.Api.V2010.CallInstance account_sid=AC48441c785c05232da5eca8fc91f9b26b sid=CAb609f67f1a700ee1e97c127544f44e87>,
 <Twilio.Api.V2010.CallInstance account_sid=AC48441c785c05232da5eca8fc91f9b26b sid=CA8ac9396bca502c0fec4d0a7823a3598f>,
 <Twilio.Api.V2010.CallInstance account_sid=AC48441c785c05232da5eca8fc91f9b26b sid=CAcc11dc52b6707d176cd5c46806b24d0f>,
 <Twilio.Api.V2010.CallInstance account_

In [14]:
for each in calls:
    callDict={}
    callDict['sid']=each.sid
    callDict['answered_by']=each.answered_by
    callDict['duration']=each.duration
    callDict['status']=each.status
    callDict['to']=each.to
    callDict['date_updated']=each.date_updated
    callDict['end_time']=each.end_time
    callDict['direction']=each.direction
    callList.append(callDict)
dfCallLogs = pd.DataFrame(callList)
dfCallLogs = dfCallLogs.loc[dfCallLogs['direction'] != 'inbound']

In [15]:
dfCallLogs

,answered_by,date_updated,direction,duration,end_time,sid,status,to
0,machine_end_silence,2018-10-04 20:55:05+00:00,outbound-api,70,2018-10-04 20:55:05+00:00,CA20c550d0ca2b8c73e15d1d991805721f,completed,+12092388184
1,machine_end_silence,2018-10-04 20:54:57+00:00,outbound-api,58,2018-10-04 20:54:57+00:00,CA15a8f261e0d3b9addefba0832d98d885,completed,+13129385217
2,machine_end_silence,2018-10-04 20:54:49+00:00,outbound-api,61,2018-10-04 20:54:49+00:00,CA3acaca67e52c3e0be3aca5fba3ab0b28,completed,+13129385308
3,machine_end_silence,2018-10-04 20:54:51+00:00,outbound-api,66,2018-10-04 20:54:51+00:00,CA1a5c4c0868e872c27fa1c41fb28bbac8,completed,+12092388184
4,machine_end_silence,2018-10-04 20:54:42+00:00,outbound-api,54,2018-10-04 20:54:42+00:00,CA7501765ed631f000261280e2765633f6,completed,+13129385217
5,machine_end_silence,2018-10-04 20:54:43+00:00,outbound-api,58,2018-10-04 20:54:43+00:00,CAb609f67f1a700ee1e97c127544f44e87,completed,+13129385308
6,None,2018-10-04 20:54:45+00:00,outbound-api,0,2018-10-04 20:54:45+00:00,CA8ac9396bca502c0fec4d0a7823a3598f,canceled,+12092388184
7,machine_end_silence,2018-10-04 20:54:47+00:00,outbound-api,58,2018-10-04 20:54:47+00:00,CAcc11dc52b6707d176cd5c46806b24d0f,completed,+13129385217
8,machine_end_silence,2018-10-04 20:54:43+00:00,outbound-api,61,2018-10-04 20:54:43+00:00,CA9f75eb28438da9b5e007fb433f9b0d79,completed,+13129385308
9,machine_end_silence,2018-10-04 20:54:55+00:00,outbound-api,69,2018-10-04 20:54:55+00:00,CA977e5acd753698ee397171f9dbb21800,completed,+12092388184


In [1]:
s = "BINGO"
print(" ".join(s))

B I N G O


In [1]:
import re

In [5]:
s = "312-938-5359"

In [3]:
p=re.findall("\d+", s)

In [4]:
p

['312', '938', '5359']

In [8]:
digit = lambda x: int(filter(str.isdigit, x) or 0)

In [9]:
digit(s)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'filter'

In [14]:
t='+1'

In [15]:
t=t+''.join(c for c in s if c.isdigit())

In [16]:
t

'+13129385359'

In [ ]:
t1="08-Nov-2018 05:19:52 PM"

In [9]:
from_lastexetime = datetime.strptime(str("08-Nov-2018 05:19:52 PM"), "%d-%b-%Y %I:%M:%S %p")

In [10]:
print(from_lastexetime)

2018-11-08 17:19:52


In [11]:
from_CST = from_lastexetime.astimezone(pytz.timezone('US/Central')).strftime("%d-%b-%Y %I:%M:%S %p")

In [12]:
from_CST

'08-Nov-2018 05:19:52 PM'

In [20]:
 hours = divmod(5008.587425, 60)[0]

In [21]:
hours

83.0

In [15]:
end_time=from_lastexetime.replace(tzinfo=pytz.utc)

In [17]:
print(end_time)

2018-11-08 17:19:52+00:00
